# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
weather_data = pd.read_csv("../output_data/weather_output.csv")
weather_data

,Unnamed: 0,Date,City,Country,Latitude,Longitude,Humidity(%),Cloudiness(%),Max Temp(F),Wind Speed(mph)
0,0,1644816152,Mar del Plata,AR,-38.0023,-57.5575,77,0,59.88,5.75
1,1,1644816115,Ushuaia,AR,-54.8000,-68.3000,81,75,38.86,8.05
2,2,1644816152,Ouango,CF,4.3133,22.5552,38,77,71.53,1.50
3,3,1644816152,Saskylakh,RU,71.9167,114.0833,95,21,-13.95,4.23
4,4,1644816152,Balkanabat,TM,39.5108,54.3671,60,8,38.71,4.47
...,...,...,...,...,...,...,...,...,...,...
566,566,1644816264,Sampit,ID,-2.5333,112.9500,57,99,89.58,2.26
567,567,1644816264,Novy Urengoy,RU,66.0833,76.6333,99,89,-12.23,8.41
568,568,1644816265,Pangnirtung,CA,66.1451,-65.7125,95,80,-33.84,4.97
569,569,1644816265,Beipiao,CN,41.7919,120.7792,56,95,15.03,16.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_data["Humidity(%)"].astype(float)

In [22]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
sum_weather_df = weather_data

sum_weather_df = sum_weather_df[(sum_weather_df["Max Temp(F)"] > 70)
                              & (sum_weather_df["Max Temp(F)"] < 80)
                               & (sum_weather_df["Cloudiness(%)"] == 0)
                             & (sum_weather_df["Wind Speed(mph)"] < 10)]
sum_weather_df

,Unnamed: 0,Date,City,Country,Latitude,Longitude,Humidity(%),Cloudiness(%),Max Temp(F),Wind Speed(mph)
80,80,1644816167,Along,IN,28.1667,94.7667,24,0,70.39,1.07
118,118,1644816174,Lüderitz,NaN,-26.6481,15.1594,72,0,70.07,8.19
147,147,1644816179,Tura,IN,25.5198,90.2201,35,0,73.36,1.01
254,254,1644816201,Jati,PK,24.3539,68.2671,49,0,71.58,6.51
263,263,1644816202,Vila Velha,BR,-20.3297,-40.2925,95,0,75.15,5.75
277,277,1644816205,Rānāpur,IN,22.6500,74.5333,25,0,76.17,8.70
374,374,1644816224,Sharjah city,AE,25.3573,55.4033,50,0,71.58,6.91
479,479,1644816249,Sālūmbar,IN,24.1333,74.0500,23,0,71.08,4.34
549,549,1644816261,La Palma,US,33.8464,-118.0467,29,0,73.11,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Hotel Map
hotel_df = sum_weather_df

# Set parameters to search hotels with 5000 meters
params = {
    "key": g_key,
    "radius": 5000,
    "keyword": "hotel",
    }

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
     # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Create url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError  ):
        print("Result not found... Skipping.")
        
    print("------------")

Retrieving Results for Index 80: Along.
Closest hotel is Hotel Toshi Palace : Dine Dwell Drink.
------------
Retrieving Results for Index 118: Lüderitz.


C:\Users\PC\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\PC\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is LÜDERITZ NEST HOTEL.
------------
Retrieving Results for Index 147: Tura.
Closest hotel is Hotel Polo Orchid.
------------
Retrieving Results for Index 254: Jati.
Closest hotel is Temra Hotel Jati.
------------
Retrieving Results for Index 263: Vila Velha.
Closest hotel is Hotel Golden Tulip Porto Vitória.
------------
Retrieving Results for Index 277: Rānāpur.
Closest hotel is My Indraprasth Tourist Homestay.
------------
Retrieving Results for Index 374: Sharjah city.
Closest hotel is Sheraton Sharjah Beach Resort & Spa.
------------
Retrieving Results for Index 479: Sālūmbar.
Closest hotel is GOKUL PALACE.
------------
Retrieving Results for Index 549: La Palma.
Closest hotel is Days Inn by Wyndham Buena Park.
------------


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))